In [2]:
import os
import sys
import pickle
import shutil
import torch

sys.path.append('../')

from accelerate import Accelerator, notebook_launcher
from dataset import EpisodeDataset, collate_fn
from model import Config, GPTModel
from trainer import train_model, validate_model
from torch.utils.data import DataLoader

In [3]:
token_to_idx = {(i, j): i * 7 + j + 1 for i in range(6) for j in range(7)}

In [4]:
token_to_idx['<pad>'] = 0  # Padding token

vocab_size = 43
block_size = 42 # Honestly this could probably be whatever
embed_size = 512
num_heads = 8
num_layers = 8
dropout = 0.1

In [5]:
#path = '/Users/wizard/Downloads/TLSAV/Datasets'
path = r'C:\Users\wmasi\Documents\Q-learning-gridworld\TLSAV\Datasets\Connect4\\'

In [6]:
with open(os.path.join(path, '1\training_data\training_games_125000.pkl'), 'rb') as f:
    agent1 = pickle.load(f)
with open(os.path.join(path, '2\training_data\training_games_125000.pkl'), 'rb') as f:
    agent2 = pickle.load(f)
with open(os.path.join(path, '3\training_data\training_games_125000.pkl'), 'rb') as f:
    agent3 = pickle.load(f)
with open(os.path.join(path, '4\training_data\training_games_125000.pkl'), 'rb') as f:
    agent4 = pickle.load(f)
with open(os.path.join(path, '5\training_data\training_games_125000.pkl'), 'rb') as f:
    agent5 = pickle.load(f)
with open(os.path.join(path, '6\training_data\training_games_125000.pkl'), 'rb') as f:
    agent6 = pickle.load(f)
with open(os.path.join(path, '7\training_data\training_games_125000.pkl'), 'rb') as f:
    agent7 = pickle.load(f)
with open(os.path.join(path, '8\training_data\training_games_125000.pkl'), 'rb') as f:
    agent8 = pickle.load(f)

In [7]:
train_ratio = 0.8
valid_ratio = 0.1

d1 = len(agent1)
d2 = len(agent2)
d3 = len(agent3)
d4 = len(agent4)
d5 = len(agent5)
d6 = len(agent6)
d7 = len(agent7)
d8 = len(agent8)

train1 = agent1[:int(train_ratio * d1)]
valid1 = agent1[int(train_ratio * d1):int((train_ratio + valid_ratio) * d1) ]
test1 = agent1[int((train_ratio + valid_ratio) * d1): ]

train2 = agent2[:int(train_ratio * d2)]
valid2 = agent2[int(train_ratio * d2):int((train_ratio + valid_ratio) * d2) ]
test2 = agent2[int((train_ratio + valid_ratio) * d2): ]

train3 = agent3[:int(train_ratio * d3)]
valid3 = agent3[int(train_ratio * d3):int((train_ratio + valid_ratio) * d3)]
test3 = agent3[int((train_ratio + valid_ratio) * d3):]

train4 = agent4[:int(train_ratio * d4)]
valid4 = agent4[int(train_ratio * d4):int((train_ratio + valid_ratio) * d4)]
test4 = agent4[int((train_ratio + valid_ratio) * d4):]

train5 = agent5[:int(train_ratio * d5)]
valid5 = agent5[int(train_ratio * d5):int((train_ratio + valid_ratio) * d5)]
test5 = agent5[int((train_ratio + valid_ratio) * d5):]

train6 = agent6[:int(train_ratio * d6)]
valid6 = agent6[int(train_ratio * d6):int((train_ratio + valid_ratio) * d6)]
test6 = agent6[int((train_ratio + valid_ratio) * d6):]

train7 = agent7[:int(train_ratio * d7)]
valid7 = agent7[int(train_ratio * d7):int((train_ratio + valid_ratio) * d7)]
test7 = agent7[int((train_ratio + valid_ratio) * d7):]

train8 = agent8[:int(train_ratio * d8)]
valid8 = agent8[int(train_ratio * d8):int((train_ratio + valid_ratio) * d8)]
test8 = agent8[int((train_ratio + valid_ratio) * d8):]

In [8]:
train = train1 + train2 + train3 + train4 + train5 + train6 + train7 + train8
valid = valid1 + valid2 + valid3 + valid4 + valid5 + valid6 + valid7 + valid8
test = test1 + test2 + test3 + test4 + test5 + test6 + test7 + test8

print(len(train))
print(len(valid))
print(len(test))

1588602
198575
198577


In [9]:
train_dataset = EpisodeDataset(train, token_to_idx)
valid_dataset = EpisodeDataset(valid, token_to_idx)
test_dataset = EpisodeDataset(test, token_to_idx)

In [10]:
def train_main(save_directory = None, epochs = 15):
    
    accelerator = Accelerator()

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
    valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

    config = Config(vocab_size, block_size, n_layer=num_layers, n_head=num_layers, n_embd=embed_size)
    model = GPTModel(config)

    optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = epochs)
    
    train_loader, valid_loader, model, scheduler, optimizer = accelerator.prepare(train_loader, valid_loader, model, scheduler, optimizer)

    epoch = 0

    model_path = None
    min_loss = 1e10
    
    for epoch in range(epochs):
        accelerator.print(f'Epoch {epoch}')

        train_model(model, train_loader, optimizer, accelerator)
        valid_loss = validate_model(model, valid_loader, accelerator)
        scheduler.step()

        if accelerator.is_main_process:
            print(f'Validation Loss: {valid_loss:.8f}')

            model_save_path = f"Model_{epoch+1}.pth"
            accelerator.save(accelerator.unwrap_model(model).state_dict(), model_save_path)

            if valid_loss < min_loss:
                min_loss = valid_loss
                model_path = model_save_path

        accelerator.wait_for_everyone()

    if accelerator.is_main_process:
        shutil.copy(model_path, save_directory)


In [12]:
notebook_launcher(train_main, (os.path.join(path, 'best_model'), 15), num_processes = 4)

Launching training on 4 GPUs.
Epoch 0


Training: 100%|██████████| 12411/12411 [21:04<00:00,  9.81it/s]





Training Loss: 0.5037913918495178
Validation Loss: 0.47267327
Epoch 1

Training:   0%|          | 0/12411 [00:00<?, ?it/s]

Training: 100%|██████████| 12411/12411 [21:05<00:00,  9.80it/s]


Training: 100%|██████████| 12411/12411 [21:05<00:00,  9.80it/s]


...

Training: 100%|██████████| 12411/12411 [21:02<00:00,  9.83it/s]





Training Loss: 0.4636186361312866
Validation Loss: 0.46616063
Epoch 14

Training:   0%|          | 0/12411 [00:00<?, ?it/s]

Training: 100%|██████████| 12411/12411 [21:02<00:00,  9.83it/s]





Training Loss: 0.46427425742149353
Validation Loss: 0.46617603
